In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-03


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\df_OHLCV_{date_str}_clean.parquet'
SOURCE_PATH = '..\data\df_finviz_n_ratios_merged.parquet'
PARQUET_PATH_COVARIANCE = '..\data\df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = '..\data\df_corr_emv_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (385536, 9)
Loaded df_finviz shape: (1536, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1536


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (251, 1536)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG   AIRR     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
2024-04-04  140.30  35.55  13.83  85.33  168.03  162.00  158.84  107.91  86.62  20.59  32.14  95.88  326.43  106.54  101.39  51.44  487.11  53.84  188.82  60.93  236.41  244.81  6.09  70.60  5.77  59.94  80.85  85.31  17.33  123.66  82.64  32.11  117.87  93.56  14.47  8.33  75.01  65.39  192.93  176.65  238.31  105.99  69.77  118.98  83.34  314.95  41.76  167.85  129.53  37.32  151.48  80.85  116.50  13.46  201.69  8.73  165.83  178.17  260.16  34.64  44.28  423.39  184.07  18.07  180.00  156.79  72.19  26.99  341.18  312.79  85.66  34.17  231.24  38.81  57.04  109.55  72.78  230.01  75.27  28.96  18.59  116.59  128.79  386.88  68.54  47.27  122.01  31.22  15.51  947.45  147.65  318.02  2.25  10.61  50.50  113.27  140.11  22.45  3.00  174.52  61.18  62.84  56.65  130.16  62.33  25.10  86.63  89.70  213.47  119.34  117.65  306.83  217.86  63.47  33.11  263.93  49.10  65.99  3085.04  183.29  70.42  36.03  144.19  65.13  39.63  166.95  41.52  45.98  64.75  2.6246  56.43  57.33  28.15   
2024-04-05  143.06  36.08  13.76  89.58  168.78  164.02  161.77  108.98  90.24  20.49  32.36  96.20  328.93  107.38  101.35  51.60  485.12  53.65  191.58  61.15  239.70  244.74  6.24  70.43  5.81  61.34  80.84  86.68  17.27  123.89  83.68  32.87  118.16  93.10  14.79  8.34  76.08  66.45  194.78  177.15  241.97  106.24  71.34  121.27  84.52  317.58  42.48  169.41  130.80  37.23  153.56  81.39  118.59  13.45  206.11  8.77  170.42  181.04  261.96  34.95  44.16  429.23  183.94  18.09  185.07  158.46  74.40  26.95  341.87  316.99  86.05  34.47  232.77  39.29  58.17  113.09  74.79  232.33  74.73  29.29  18.78  118.56  130.69  387.03  67.43  47.12  124.82  31.87  15.34  973.43  149.88  319.46  2.20  10.79  51.73  113.37  140.10  23.27  3.00  175.40  61.40  63.16  56.71  132.33  62.89  25.87  87.50  90.15  214.42  120.58  115.83  309.90  220.70  64.15  33.02  271.30  49.76  66.09  3104.94  183.14  70.14  36.22  144.67  65.61  39.97  169.20  41.51  46.23  65.18  2.5884  56.59  57.63  28.10   
2024

In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (251, 1536)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG   AIRR     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
2024-04-04  140.30  35.55  13.83  85.33  168.03  162.00  158.84  107.91  86.62  20.59  32.14  95.88  326.43  106.54  101.39  51.44  487.11  53.84  188.82  60.93  236.41  244.81  6.09  70.60  5.77  59.94  80.85  85.31  17.33  123.66  82.64  32.11  117.87  93.56  14.47  8.33  75.01  65.39  192.93  176.65  238.31  105.99  69.77  118.98  83.34  314.95  41.76  167.85  129.53  37.32  151.48  80.85  116.50  13.46  201.69  8.73  165.83  178.17  260.16  34.64  44.28  423.39  184.07  18.07  180.00  156.79  72.19  26.99  341.18  312.79  85.66  34.17  231.24  38.81  57.04  109.55  72.78  230.01  75.27  28.96  18.59  116.59  128.79  386.88  68.54  47.27  122.01  31.22  15.51  947.45  147.65  318.02  2.25  10.61  50.50  113.27  140.11  22.45  3.00  174.52  61.18  62.84  56.65  130.16  62.33  25.10  86.63  89.70  213.47  119.34  117.65  306.83  217.86  63.47  33.11  263.93  49.10  65.99  3085.04  183.29  70.42  36.03  144.19  65.13  39.63  166.95  41.52  45.98  64.75  2.6246  56.43  57.33  28.15   
2024-04-05  143.06  36.08  13.76  89.58  168.78  164.02  161.77  108.98  90.24  20.49  32.36  96.20  328.93  107.38  101.35  51.60  485.12  53.65  191.58  61.15  239.70  244.74  6.24  70.43  5.81  61.34  80.84  86.68  17.27  123.89  83.68  32.87  118.16  93.10  14.79  8.34  76.08  66.45  194.78  177.15  241.97  106.24  71.34  121.27  84.52  317.58  42.48  169.41  130.80  37.23  153.56  81.39  118.59  13.45  206.11  8.77  170.42  181.04  261.96  34.95  44.16  429.23  183.94  18.09  185.07  158.46  74.40  26.95  341.87  316.99  86.05  34.47  232.77  39.29  58.17  113.09  74.79  232.33  74.73  29.29  18.78  118.56  130.69  387.03  67.43  47.12  124.82  31.87  15.34  973.43  149.88  319.46  2.20  10.79  51.73  113.37  140.10  23.27  3.00  175.40  61.40  63.16  56.71  132.33  62.89  25.87  87.50  90.15  214.42  120.58  115.83  309.90  220.70  64.15  33.02  271.30  49.76  66.09  3104.94  183.14  70.14  36.22  144.67  65.61  39.97  169.20  41.51  46.23  65.18  2.5884  56.59  57.63  28.10   
2024

df_close_returns shape: (250, 1536)


Symbol             A        AA       AAL      AAON      AAPL      ABBV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG      AIRR       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-05  0.019672  0.014909 -0.005061  0.049807  0.004463  0.012469  0.018446  0.009916  0.041792 -0.004857  0.006845  0.003338  0.007659  0.007884 -0.000395  0.003110 -0.004085 -0.003529  0.014617  0.003611  0.013917 -0.000286  0.024631 -0.002408  0.006932  0.023357 -0.000124  0.016059 -0.003462  0.001860  0.012585  0.023669  0.002460 -0.004917  0.022115  0.001200  0.014265  0.016210  0.009589  0.002830  0.015358  0.002359  0.022503  0.019247  0.014159  0.008351  0.017241  0.009294  0.009805 -0.002412  0.013731  0.006679  0.017940 -0.000743  0.021915  0.004582  0.027679  0.016108  0.006919  0.008949 -0.002710  0.013793 -0.000706  0.001107  0.028167  0.010651  0.030614 -0.001482  0.002022  0.013428  0.004553  0.008780  0.006617  0.012368  0.019811  0.032314  0.027617  0.010087 -0.007174  0.011395  0.010221  0.016897  0.014753  0.000388 -0.016195 -0.003173  0.023031  0.020820 -0.010961  0.027421  0.015103  0.004528 -0.022222  0.016965  0.024356  0.000883 -0.000071  0.036526   
2024-04-08  0.002307 -0.004157  0.010174  0.032931 -0.006636 -0.001158 -0.009272 -0.006148 -0.003657 -0.003416  0.019778  0.003326 -0.003618  0.002049  0.001875  0.004651 -0.001732  0.019758  0.009918 -0.004415  0.007301  0.004699  0.019231  0.008661  0.022375 -0.013694  0.003711 -0.004153  0.012160  0.006780 -0.003227  0.037724 -0.007955 -0.000967  0.002028  0.003597  0.001709  0.001956  0.003594  0.007790  0.000703 -0.003953 -0.019905  0.066711 -0.011358  0.007809  0.002354 -0.002656  0.004128  0.004835  0.016150  0.003440  0.002024 -0.005948  0.005725 -0.005701 -0.003051 -0.007181 -0.002749  0.015737 -0.000679  0.002213 -0.010275  0.024323  0.000648  0.003597 -0.000672  0.002226  0.006523  0.002650 -0.006508 -0.019727 -0.004382  0.007126 -0.006533  0.012114  0.043589  0.003745  0.017931  0.023216 -0.000532  0.017122  0.007499 -0.000827  0.064660  0.020798  0.035491  0.009727 -0.053455  0.003226 -0.006672 -0.002817  0.031818 -0.014829  0.001160  0.000882 -0.008637 -0.016760   
2024-04-09  0.020364  0.

In [8]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1536, 1536)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE           ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG      AIRR       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
A       0.000391  0.000583  0.000604  0.000542  0.000447  0.000091  0.000368  0.000027  0.000092 -0.000053  0.000289  0.000233  0.000271  0.000220  0.000039  0.000129  0.000302 -2.754004e-07  0.000500  0.000033  0.000076  0.000252  0.000252 -0.000017  0.000350  0.000016 -0.000047  0.000292  0.000235  0.000077  0.000057  0.000935  0.000518 -0.000035 -0.000004  0.000129  0.000160  0.000410  0.000450  0.000166  0.000031  0.000265  0.000712  0.000397  0.000223  0.000414  0.000580 -0.000001  0.000236  0.000505  0.000339  0.000438  0.000336  0.000140  0.000438  0.000130  0.000469  0.000345 -0.000017  0.000133  0.000176  0.000424 -0.000180 -0.000161  0.000453  0.000341  0.000589  0.000066  0.000158  0.000029  0.000202  0.000731  0.000213  0.000374  0.000436  0.000637  0.000444  0.000168  0.000476  0.000390  0.000213  0.000320  0.000778  0.000119  0.000384  0.000516  0.000547  0.000175  0.000753  0.000368  0.000204  0.000130  0.000550  0.000442  0.000529  0.000026  0.000135  0.000093   
AA      0.000583  0.001494  0.001046  0.000931  0.000864  0.000069  0.000793 -0.000082  0.000148 -0.000133  0.000734  0.000428  0.000496  0.000442  0.000041  0.000273  0.000612 -7.522136e-05  0.000898 -0.000019  0.000102  0.000579  0.000550 -0.000080  0.000684 -0.000008 -0.000216  0.000618  0.000488  0.000106  0.000111  0.001988  0.001037 -0.000062  0.000021  0.000193  0.000235  0.000853  0.000968  0.000384  0.000006  0.000484  0.001573  0.000903  0.000235  0.000825  0.001187  0.000054  0.000358  0.001020  0.000381  0.000918  0.000743  0.000331  0.000874  0.000214  0.001043  0.000684 -0.000175  0.000195  0.000390  0.000867 -0.000459 -0.000303  0.000944  0.000724  0.001223  0.000137  0.000358  0.000042  0.000373  0.001586  0.000413  0.000737  0.000921  0.001437  0.001492  0.000342  0.000989  0.000887  0.000388  0.000575  0.001652  0.000105  0.000831  0.001074  0.001041  0.000387  0.001509  0.000796  0.000394  0.000284  0.000758  0.000982  0.001110 -0.000013  0.000188  0.000296   
AAL     0.000604  0.0010

corr_emv_matrix shape: (1536, 1536)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG      AIRR       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       1.000000  0.763112  0.826976  0.723035  0.766477  0.342254  0.639222  0.105241  0.387231 -0.170210  0.564768  0.735571  0.633818  0.825752  0.450026  0.702986  0.574249 -0.001783  0.833905  0.111229  0.373713  0.682624  0.601534 -0.092645  0.770329  0.063895 -0.191865  0.765081  0.491070  0.370723  0.318787  0.716968  0.697331 -0.621746 -0.013596  0.544939  0.568339  0.816449  0.745505  0.523129  0.155006  0.564847  0.653107  0.676570  0.540462  0.813126  0.780327 -0.006292  0.699950  0.735036  0.489124  0.751706  0.669306  0.522139  0.803528  0.487338  0.692636  0.822980 -0.073976  0.578662  0.703040  0.800188 -0.551423 -0.544591  0.754885  0.658069  0.735993  0.744141  0.667416  0.189432  0.703576  0.758005  0.742685  0.764061  0.727821  0.774247  0.299762  0.455096  0.779954  0.658036  0.733451  0.762606  0.794356  0.293562  0.609420  0.720062  0.706555  0.530812  0.829180  0.727799  0.374440  0.302045  0.491791  0.698375  0.780658  0.155801  0.610896  0.242036  0.634296   
AA      0.763112  1.000000  0.733049  0.635283  0.757926  0.132934  0.704856 -0.161142  0.319120 -0.217948  0.732559  0.690368  0.592616  0.847216  0.238661  0.761236  0.595375 -0.249163  0.766217 -0.033883  0.255144  0.801335  0.671027 -0.217672  0.771244 -0.016176 -0.451333  0.829273  0.521683  0.261683  0.317127  0.780069  0.714415 -0.560070  0.034423  0.417134  0.425056  0.868296  0.820390  0.617009  0.015829  0.527892  0.737600  0.787332  0.292366  0.828196  0.817049  0.116124  0.544430  0.759467  0.281045  0.806563  0.756435  0.631245  0.819985  0.411927  0.787656  0.833616 -0.388699  0.436188  0.798118  0.836574 -0.720462 -0.525078  0.803745  0.714305  0.781419  0.787561  0.770441  0.140715  0.663392  0.840943  0.735294  0.769975  0.785915  0.892618  0.515700  0.473587  0.827862  0.765491  0.681973  0.701052  0.862175  0.131815  0.675623  0.767092  0.688065  0.601844  0.849723  0.806328  0.369090  0.338750  0.346242  0.792520  0.837270 -0.040352  0.436830  0.396178  0.634256   


In [9]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE)
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION)
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, A to ZTS
Columns: 1536 entries, A to ZTS
dtypes: float64(1536)
memory usage: 18.0+ MB


None

corr_emv_matrix parqueted to ..\data\df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1536 entries, A to ZTS
Columns: 1536 entries, A to ZTS
dtypes: float64(1536)
memory usage: 18.0+ MB


None